In [4]:
%pip install langchain
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# %pip install -qU langchain-openai
%pip install langchain-openai
%pip install langchain-community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import getpass
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Load environment variables from .env file
load_dotenv()

open_ai_key = os.getenv('OPEN_AI_KEY')
open_ai_org = os.getenv('OPEN_AI_ORG')

model = ChatOpenAI(
    openai_api_key=open_ai_key,
    openai_organization=open_ai_org,
    max_tokens=1000,
    model="gpt-4"
)

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]



# Managing a Chat Bot History

In [11]:
import langchain_core.messages as messages
print(dir(messages))

['AIMessage', 'AIMessageChunk', 'AnyMessage', 'BaseMessage', 'BaseMessageChunk', 'ChatMessage', 'ChatMessageChunk', 'FunctionMessage', 'FunctionMessageChunk', 'HumanMessage', 'HumanMessageChunk', 'InvalidToolCall', 'MessageLikeRepresentation', 'SystemMessage', 'SystemMessageChunk', 'ToolCall', 'ToolCallChunk', 'ToolMessage', 'ToolMessageChunk', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_message_from_dict', 'ai', 'base', 'chat', 'convert_to_messages', 'filter_messages', 'function', 'get_buffer_string', 'human', 'merge_content', 'merge_message_runs', 'message_chunk_to_message', 'message_to_dict', 'messages_from_dict', 'messages_to_dict', 'system', 'tool', 'trim_messages', 'utils']


In [12]:
from langchain_core.messages import SystemMessage, trim_messages

In [13]:
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [15]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [16]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."

In [17]:
chain.config_schema

<bound method Runnable.config_schema of RunnableAssign(mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], template='You are a helpful assistant. Answer all questions to the best of your ability in {language}.')), MessagesPlaceholder(variable_name='messages')])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10b912030>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10b76a330>, model_name='gpt-4', openai_api_key=SecretStr(

In [18]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [19]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

NameError: name 'store' is not defined